In [1]:
# imports
import os
import sys
import time
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(10)

%load_ext autoreload
%autoreload 2

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# helper method to define model
def deepnn(x):
    with tf.variable_scope('model'):
        
        x_image = tf.reshape(x, [-1, 28, 28, 1])
    #     x_image = tf.image.resize_image_with_cop_or_pad(x_image, target_height=32, target_width=32)

        W_conv1 = tf.get_variable("weight_1", shape=[5, 5, 1, 6], initializer=tf.truncated_normal_initializer(stddev=0.1))
        b_conv1 = tf.get_variable("bias_1", shape=[6], initializer=tf.constant_initializer(0.1))
        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

        h_pool1 = max_pool_2x2(h_conv1)

        W_conv2 = tf.get_variable("weight_2", shape=[5, 5, 6, 16], initializer=tf.truncated_normal_initializer(stddev=0.1))
        b_conv2 = tf.get_variable("bias_2", shape=[16], initializer=tf.constant_initializer(0.1))
        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

        h_pool2 = max_pool_2x2(h_conv2)

        h_pool_flatten = tf.reshape(h_pool2, (-1, 7*7*16))

        W_fc1 = tf.get_variable("weight_3", shape=[7*7*16, 120], initializer=tf.truncated_normal_initializer(stddev=0.1))
        b_fc1 = tf.get_variable("bias_3", shape=[120], initializer=tf.constant_initializer(0.1))
        h_fc1 = tf.matmul(h_pool_flatten, W_fc1) + b_fc1

        W_fc2 = tf.get_variable("weight_4", shape=[120, 10], initializer=tf.truncated_normal_initializer(stddev=0.1))
        b_fc2 = tf.get_variable("bias_4", shape=[10],initializer=tf.constant_initializer(0.1))
        y_conv = tf.matmul(h_fc1, W_fc2) + b_fc2
        return y_conv


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [3]:
#load graph and data and run training

tf.reset_default_graph()

# defining TF model
# reading data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# defining TF model
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
y_conv = deepnn(x)

# print(y_conv.get_shape().as_list())
# print(y_.get_shape().as_list())

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_conv, labels = y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

saver = tf.train.Saver(max_to_keep=100, keep_checkpoint_every_n_hours=1)
sess= tf.Session()

# saver.restore(sess, './cifar_model/model_chkpt-6370')
init_op = tf.group(tf.initialize_all_variables(), tf.initialize_local_variables())
# init_op = tf.group(tf.initialize_local_variables())
sess.run(init_op)

# qr = tf.train.QueueRunner(queue=queue, enqueue_ops=[enqueue_op] * 1, close_op = queue_close)
# tf.train.add_queue_runner(qr)
with sess.as_default():
    for i in range(10000):
        batch = mnist.train.next_batch(50)
        if i%10 == 0:
            saver.save(sess, './mnist_model/model_chkpt', global_step=i)
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1]})
            print("step %d, training accuracy %g"%(i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})
        
    x_feed = mnist.test.images.reshape(-1, 784)[:100]
    y_feed = mnist.test.labels.reshape(-1, 10)[:100]
    print("test accuracy %g"%accuracy.eval(feed_dict={x: x_feed, y_: y_feed, 
                                                      }))

    test_x = mnist.test.images.reshape(-1, 784)[0]
    test_y = mnist.test.images.reshape(-1, 10)[0]
    feed_dict = {x: test_x, y:test_y}

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.local_variables_initializer` instead.
step 0, training accuracy 0.08
step 10, training accuracy 0.1
step 20, training accuracy 0.04
step 30, training accuracy 0.18
step 40, training accuracy 0.2
step 50, training accuracy 0.22
step 60, training accuracy 0.26
step 70, training accuracy 0.42
step 80, training accuracy 0.42
step 90, training accuracy 0.54
step 100, training accuracy 0.5
step 110, training accuracy 0.46
step 120, training accuracy 0.42
step 130, training accuracy 0.62
step 140, training acc

step 2120, training accuracy 0.96
step 2130, training accuracy 0.96
step 2140, training accuracy 0.96
step 2150, training accuracy 0.94
step 2160, training accuracy 0.96
step 2170, training accuracy 0.96
step 2180, training accuracy 1
step 2190, training accuracy 0.96
step 2200, training accuracy 0.98
step 2210, training accuracy 0.98
step 2220, training accuracy 0.86
step 2230, training accuracy 0.88
step 2240, training accuracy 0.92
step 2250, training accuracy 1
step 2260, training accuracy 1
step 2270, training accuracy 0.98
step 2280, training accuracy 0.96
step 2290, training accuracy 0.92
step 2300, training accuracy 0.96
step 2310, training accuracy 0.92
step 2320, training accuracy 0.94
step 2330, training accuracy 1
step 2340, training accuracy 0.96
step 2350, training accuracy 0.9
step 2360, training accuracy 0.92
step 2370, training accuracy 0.94
step 2380, training accuracy 0.94
step 2390, training accuracy 0.94
step 2400, training accuracy 0.96
step 2410, training accurac

step 4570, training accuracy 0.98
step 4580, training accuracy 0.98
step 4590, training accuracy 1
step 4600, training accuracy 0.96
step 4610, training accuracy 0.96
step 4620, training accuracy 0.98
step 4630, training accuracy 0.98
step 4640, training accuracy 0.98
step 4650, training accuracy 0.98
step 4660, training accuracy 0.98
step 4670, training accuracy 1
step 4680, training accuracy 0.96
step 4690, training accuracy 0.94
step 4700, training accuracy 0.96
step 4710, training accuracy 0.94
step 4720, training accuracy 1
step 4730, training accuracy 1
step 4740, training accuracy 0.96
step 4750, training accuracy 0.96
step 4760, training accuracy 0.98
step 4770, training accuracy 1
step 4780, training accuracy 0.96
step 4790, training accuracy 0.98
step 4800, training accuracy 0.98
step 4810, training accuracy 0.94
step 4820, training accuracy 0.96
step 4830, training accuracy 0.96
step 4840, training accuracy 1
step 4850, training accuracy 0.96
step 4860, training accuracy 0.9

step 7040, training accuracy 1
step 7050, training accuracy 0.98
step 7060, training accuracy 0.94
step 7070, training accuracy 1
step 7080, training accuracy 0.98
step 7090, training accuracy 0.98
step 7100, training accuracy 0.98
step 7110, training accuracy 0.98
step 7120, training accuracy 0.98
step 7130, training accuracy 1
step 7140, training accuracy 1
step 7150, training accuracy 0.98
step 7160, training accuracy 0.98
step 7170, training accuracy 0.96
step 7180, training accuracy 1
step 7190, training accuracy 0.98
step 7200, training accuracy 0.96
step 7210, training accuracy 1
step 7220, training accuracy 0.98
step 7230, training accuracy 0.94
step 7240, training accuracy 0.96
step 7250, training accuracy 0.94
step 7260, training accuracy 0.98
step 7270, training accuracy 0.94
step 7280, training accuracy 0.94
step 7290, training accuracy 1
step 7300, training accuracy 1
step 7310, training accuracy 1
step 7320, training accuracy 1
step 7330, training accuracy 0.98
step 7340,

step 9540, training accuracy 1
step 9550, training accuracy 0.98
step 9560, training accuracy 1
step 9570, training accuracy 0.98
step 9580, training accuracy 0.98
step 9590, training accuracy 1
step 9600, training accuracy 1
step 9610, training accuracy 0.96
step 9620, training accuracy 0.98
step 9630, training accuracy 0.98
step 9640, training accuracy 0.96
step 9650, training accuracy 0.96
step 9660, training accuracy 0.98
step 9670, training accuracy 0.92
step 9680, training accuracy 0.98
step 9690, training accuracy 0.98
step 9700, training accuracy 0.96
step 9710, training accuracy 1
step 9720, training accuracy 0.98
step 9730, training accuracy 0.98
step 9740, training accuracy 0.98
step 9750, training accuracy 1
step 9760, training accuracy 0.94
step 9770, training accuracy 0.98
step 9780, training accuracy 1
step 9790, training accuracy 0.98
step 9800, training accuracy 0.96
step 9810, training accuracy 0.98
step 9820, training accuracy 0.94
step 9830, training accuracy 0.98
s

NameError: name 'y' is not defined

In [ ]:
tf.global_variables()